# Introduction

Topic Modeling is a technique to extract the hidden topics from large volumes of text. Latent Dirichlet Allocation(LDA) is a popular algorithm for topic modeling with excellent implementations in the Python’s Gensim package. The challenge, however, is how to extract good quality of topics that are clear, segregated and meaningful. This depends heavily on the quality of text preprocessing and the strategy of finding the optimal number of topics. This tutorial attempts to tackle both of these problems.

# Prerequisites 

– Download nltk stopwords and spacy model
We will need the stopwords from NLTK and spacy’s en model for text pre-processing. Later, we will be using the spacy model for lemmatization.

Lemmatization is nothing but converting a word to its root word. For example: the lemma of the word ‘machines’ is ‘machine’. Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

In [1]:
# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/ping/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Import Packages


In [2]:
#pip install spacy

In [3]:
#pip install pyLDAvis

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

2022-12-05 22:53:23.462826: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 22:53:24.161860: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-05 22:53:24.161970: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-05 22:53:24.161976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# What does LDA do?

LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.


Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

# Prepare Stopwords

We have already downloaded the stopwords. Let’s import them and make it available in stop_words.

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Data

In [6]:
df = pd.read_csv("data_cleaned.csv")
df

,Unnamed: 0,Id,Title,Body,Tags
0,0,1898,Import CSV file to strongly typed data structu...,<p>What's the best way to import a CSV file in...,<c#><vb.net><file><csv><import>
1,1,1939041,Change hash without reload in jQuery,<p>I have the following code:</p>\n\n<pre><cod...,<jquery><hash><reload><fragment-identifier><wi...
2,2,2027,Pass by reference or pass by value?,"<p>When learning a new programming language, o...",<language-agnostic><oop><parameters><pass-by-r...
3,3,2056,What are MVP and MVC and what is the difference?,"<p>When looking beyond the <a href=""https://en...",<user-interface><model-view-controller><design...
4,4,2138,LINQ on the .NET 2.0 Runtime,<p>Can a LINQ enabled app run on a machine tha...,<.net><linq><.net-3.5><.net-2.0><clr>
...,...,...,...,...,...
27928,27928,37332190,Django - Login with Email,<p>I want django to authenticate users via ema...,<python><python-3.x><django><django-authentica...
27929,27929,37334668,Is there a python equivalent for RSpec to do TDD?,<p>I'm looking for a test framework like Ruby'...,<python><ruby><testing><rspec><tdd>
27930,27930,55345373,How to install GCC/G++ 8 on CentOS,<p>I'm used to install packages on Debian/Ubun...,<gcc><centos><g++><software-collections><redha...
27931,27931,55372259,How to use TabLayout with ViewPager2 in Android,<p>I want to use <code>com.google.android.mate...,<java><android><androidx><android-tablayout><a...


In [7]:
df = df[['Id', 'Title', 'Body', 'Tags']]

In [8]:
df=df.sample(frac=0.1)
df.shape

(2793, 4)

In [9]:
print(df.Tags.unique())
df.head()

['<android><input><popup><android-edittext><drawable>'
 '<python><google-drive-api><google-sheets-api><google-docs><google-docs-api>'
 '<ios><core-location><cllocationmanager><power-management><core-bluetooth>'
 ... '<javascript><html><css><hover><touch>'
 '<git><version-control><versioning><terminology><definition>'
 '<java><image><swing><screenshot><jframe>']


,Id,Title,Body,Tags
24203,14127710,Change background of EditText's error message,<p>What I want to do is change the background ...,<android><input><popup><android-edittext><draw...
8767,3287651,Download a spreadsheet from Google Drive / Wor...,<p>Can you produce a Python example of how to ...,<python><google-drive-api><google-sheets-api><...
3699,9513249,Battery drain when using CoreLocation Signific...,<p>We've released an application which runs in...,<ios><core-location><cllocationmanager><power-...
5221,27411826,Detect if checkbox is checked or unchecked in ...,<p>I want to detect if a checkbox has been che...,<javascript><jquery><angularjs><checkbox><angu...
21112,11186854,Download a file / image with AFNetworking in iOS?,<p>I thought I had it figured out but I just c...,<iphone><objective-c><ios><ipad><afnetworking>


# Remove characters
As you can see there are many emails, newline and extra spaces that is quite distracting. Let’s get rid of them using regular expressions.

In [10]:
# Convert to list
data = df.Body.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['<p>What I want to do is change the background color (set custom drawable) of '
 'a popup error message displayed after using <code>setError()</code> '
 'method.</p> <p>Currently, it looks like this:</p> <p><img '
 'src="https://i.stack.imgur.com/XmFEG.png" alt="enter image description '
 'here"></p> <p>Ive found that Android has two files:</p> <ul> '
 '<li><code>popup_inline_error.9.png</code></li> '
 '<li><code>popup_inline_above_error.9.png</code></li> </ul> <p>And youre '
 'supposed to be able to set them using two attributes:</p> <ul> '
 '<li><code>errorMessageBackground</code></li> '
 '<li><code>errorMessageAboveBackground</code></li> </ul> <p>But when I try to '
 'set them in my theme, all I get is:</p> <pre><code>&lt;item &lt;item error: '
 'Error: No resource found that matches the given name: attr '
 'errorMessageBackground. </code></pre> <p>(its the same <code>with '
 'android:errorMessageBackground</code>)</p> <p>Im putting this question here, '
 'cause Ive run out of idea

# Tokenize words and Clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['what', 'want', 'to', 'do', 'is', 'change', 'the', 'background', 'color', 'set', 'custom', 'drawable', 'of', 'popup', 'error', 'message', 'displayed', 'after', 'using', 'code', 'seterror', 'code', 'method', 'currently', 'it', 'looks', 'like', 'this', 'img', 'src', 'https', 'stack', 'imgur', 'com', 'xmfeg', 'png', 'alt', 'enter', 'image', 'description', 'here', 'ive', 'found', 'that', 'android', 'has', 'two', 'files', 'ul', 'li', 'code', 'png', 'code', 'li', 'li', 'code', 'png', 'code', 'li', 'ul', 'and', 'youre', 'supposed', 'to', 'be', 'able', 'to', 'set', 'them', 'using', 'two', 'attributes', 'ul', 'li', 'code', 'code', 'li', 'li', 'code', 'code', 'li', 'ul', 'but', 'when', 'try', 'to', 'set', 'them', 'in', 'my', 'theme', 'all', 'get', 'is', 'pre', 'code', 'lt', 'item', 'lt', 'item', 'error', 'error', 'no', 'resource', 'found', 'that', 'matches', 'the', 'given', 'name', 'attr', 'code', 'pre', 'its', 'the', 'same', 'code', 'with', 'android', 'code', 'im', 'putting', 'this', 'questio

Creating Bigram and Trigram Models
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['what', 'want', 'to', 'do', 'is', 'change', 'the', 'background', 'color', 'set', 'custom', 'drawable', 'of', 'popup', 'error', 'message', 'displayed', 'after', 'using', 'code', 'seterror', 'code', 'method', 'currently', 'it', 'looks', 'like', 'this', 'img_src_https_stack', 'imgur', 'com', 'xmfeg', 'png_alt_enter_image', 'description_here', 'ive', 'found', 'that', 'android', 'has', 'two', 'files', 'ul', 'li', 'code', 'png', 'code', 'li', 'li', 'code', 'png', 'code', 'li', 'ul', 'and', 'youre', 'supposed', 'to', 'be', 'able', 'to', 'set', 'them', 'using', 'two', 'attributes', 'ul', 'li', 'code', 'code', 'li', 'li', 'code', 'code', 'li', 'ul', 'but', 'when', 'try', 'to', 'set', 'them', 'in', 'my', 'theme', 'all', 'get', 'is', 'pre', 'code', 'lt', 'item', 'lt', 'item', 'error', 'error', 'no', 'resource', 'found', 'that', 'matches', 'the', 'given', 'name', 'attr', 'code', 'pre', 'its', 'the', 'same', 'code', 'with', 'android', 'code', 'im', 'putting', 'this', 'question', 'here', 'cause', '

# Remove Stopwords, Make Bigrams and Lemmatize

The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
#pip install spacy-download

In [18]:
from spacy_download import load_spacy

nlp = load_spacy("en_core_web_sm", exclude=["parser", "tagger"])  # Will download the model if it isn't installed yet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
#nlp = spacy.load("en_core_web_sm")
#nlp = load_spacy("en_core_web_sm", exclude=["parser", "tagger"])  # Will download the model if it isn't installed yet

nlp = load_spacy("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['want', 'change', 'background', 'color', 'set', 'custom', 'drawable', 'popup', 'error', 'message', 'display', 'use', 'code', 'seterror', 'code', 'method', 'currently', 'look', 'img_src', 'imgur', 'com', 'png_alt', 'enter_image', 'description', 've', 'find', 'android', 'file', 're', 'suppose', 'able', 'set', 'use', 'attribute', 'try', 'set', 'theme', 'get', 'code', 'error', 'error', 'resource', 'find', 'match', 'give', 'name', 'm', 'put', 'question', 've', 'run', 'idea', 'maybe', 'already', 'manage', 'strong', 'edit', 'strong', 'header', 'theme', 'm', 'use', 'code', 'resource', 'android', 'http_schema', 'android', 'com', 'android', 'style', 'name', 'theme', 'mythemename', 'code', 'strong', 'edit', 'strong', 've', 'find', 'question', 'duplicate', 'com', 'question', 'get', 'resource', 'find', 'error', 'style', 'android', 'get', 'resource', 'find', 'error', 'style', 'xml', 'strong', 'yet', 'edit', 'strong', 'know', 'problem', 'take', 'look', 'android', 'issue', 'detail', 'noreferr', 'and

# Create the Dictionary and Corpus needed for Topic Modeling

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 7), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 3), (18, 1), (19, 5), (20, 1), (21, 5), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 2), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 3), (47, 1), (48, 4), (49, 1), (50, 3), (51, 1), (52, 6), (53, 3), (54, 1), (55, 1), (56, 3), (57, 1), (58, 3), (59, 3), (60, 1), (61, 1), (62, 1)]]


In [21]:
id2word[0]

'able'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('already', 1),
  ('android', 7),
  ('attribute', 1),
  ('background', 1),
  ('change', 1),
  ('code', 5),
  ('color', 1),
  ('com', 3),
  ('currently', 1),
  ('custom', 1),
  ('d', 1),
  ('description', 1),
  ('detail', 2),
  ('display', 1),
  ('drawable', 1),
  ('duplicate', 1),
  ('edit', 3),
  ('enter_image', 1),
  ('error', 5),
  ('file', 1),
  ('find', 5),
  ('get', 3),
  ('give', 1),
  ('header', 1),
  ('http_schema', 1),
  ('i', 1),
  ('idea', 1),
  ('img_src', 1),
  ('imgur', 1),
  ('issue', 2),
  ('know', 1),
  ('look', 2),
  ('m', 2),
  ('manage', 1),
  ('match', 1),
  ('maybe', 1),
  ('message', 1),
  ('method', 1),
  ('mythemename', 1),
  ('name', 2),
  ('noreferr', 1),
  ('png_alt', 1),
  ('popup', 1),
  ('problem', 1),
  ('put', 1),
  ('question', 3),
  ('re', 1),
  ('resource', 4),
  ('run', 1),
  ('set', 3),
  ('seterror', 1),
  ('strong', 6),
  ('style', 3),
  ('suppose', 1),
  ('take', 1),
  ('theme', 3),
  ('try', 1),
  ('use', 3),
  ('ve', 3),
  ('

# Building the Topic Model

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.


You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics() as shown next.

The weights reflect how important a keyword is to that topic.

In [24]:
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.149*"self" + 0.116*"view" + 0.068*"imgur" + 0.057*"png_alt" + '
  '0.048*"description" + 0.041*"button" + 0.040*"https_stack" + '
  '0.035*"enter_image" + 0.035*"position" + 0.025*"img_src"'),
 (1,
  '0.161*"image" + 0.116*"text" + 0.075*"form" + 0.049*"input" + '
  '0.046*"number" + 0.039*"python" + 0.025*"gem" + 0.025*"rail" + 0.020*"word" '
  '+ 0.018*"scale"'),
 (2,
  '0.069*"join" + 0.045*"relationship" + 0.043*"somehow" + 0.027*"primary_key" '
  '+ 0.025*"textview" + 0.025*"car" + 0.018*"readable" + 0.017*"inflate" + '
  '0.015*"void_onclick" + 0.013*"extends_activity"'),
 (3,
  '0.073*"standard" + 0.057*"language" + 0.056*"feature" + 0.046*"template" + '
  '0.032*"hard" + 0.029*"notification" + 0.025*"useful" + 0.021*"db" + '
  '0.019*"face" + 0.019*"feel"'),
 (4,
  '0.331*"code" + 0.034*"pre" + 0.023*"type" + 0.023*"get" + 0.021*"list" + '
  '0.021*"follow" + 0.018*"call" + 0.018*"value" + 0.017*"result" + '
  '0.016*"try"'),
 (5,
  '0.077*"class" + 0.061*"return" + 0

# Compute Model Perplexity and Coherence Score

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.058472507247535

Coherence Score:  0.40761038333213595


There you have a coherence score of 0.40.

 # Visualize the topics-keywords

Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

In [26]:
from pyLDAvis import gensim_models
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensim_models.prepare(lda_model, corpus, id2word)
vis

/home/ping/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/ping/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ping/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ping/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ping/anaconda3/lib/python3.9/site-packages/past/builtins/mi

/home/ping/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/home/ping/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/ping/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/home/ping/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/ping/anaconda3/lib/python3.9/site-packages/sklearn/utils/multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, th

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.313642 -0.003242       1        1  25.581284
4      0.301139  0.166302       2        1  15.711577
18     0.287597  0.077006       3        1  15.687359
5      0.247913  0.091685       4        1  11.251689
9      0.050582 -0.267601       5        1   4.595230
13     0.101740 -0.195563       6        1   4.041094
12     0.013573 -0.099739       7        1   3.318714
14     0.023900 -0.175372       8        1   2.989425
17    -0.035226 -0.055737       9        1   2.544950
1     -0.075421  0.055922      10        1   2.242337
0     -0.061483  0.031952      11        1   2.232897
11    -0.090984  0.070254      12        1   1.763333
10    -0.072781 -0.013485      13        1   1.728405
16    -0.116087  0.049256      14        1   1.369751
3     -0.135269  0.032419      15        1   1.267265
8     -0.131217  0.058475      16        1   1.154010
15    -0.126550  0.044947      17        1   0.979330
7     -0.141533  0.043912      18        1   0.913140
19    -0.170585  0.045084      19        1   0.390895
2     -0.182950  0.043527      20        1   0.237315, topic_info=           Term          Freq         Total Category  logprob  loglift
6          code  12249.000000  12249.000000  Default  30.0000  30.0000
52       strong   4244.000000   4244.000000  Default  29.0000  29.0000
20         file   1941.000000   1941.000000  Default  28.0000  28.0000
8           com   2746.000000   2746.000000  Default  27.0000  27.0000
515       class   2044.000000   2044.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
569   os_bundle      0.025579      1.079161  Topic20  -9.9891   2.3014
7026    pyspark      0.025579      1.079166  Topic20  -9.9891   2.3014
2447    session      0.025579      1.079367  Topic20  -9.9891   2.3012
4355    mapping      0.025579      1.079192  Topic20  -9.9891   2.3013
1850    dismiss      0.025579      1.079170  Topic20  -9.9891   2.3014

[846 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.959623        able
0         3  0.038935        able
637       4  0.985881      accept
1544     17  0.964448  accomplish
134       5  0.979437     account
...     ...       ...         ...
1762     15  0.942822         wpf
133       1  0.934992       write
133       2  0.063973       write
1270     13  0.924919       xcode
61        3  0.995130         xml

[1016 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 19, 6, 10, 14, 13, 15, 18, 2, 1, 12, 11, 17, 4, 9, 16, 8, 20, 3])